## Cover page placeholder

In [ ]:
# pip install python-constraint

## Scenario 1 
### Ciara knows Python, and only has funds to hire three more people.

It won't be possible to fill all of positions with just 3 hires, so Ciara will have to take on one of the Python positions.

In [2]:
from constraint import Problem

# Define the positions and the hires with their qualifications
positions = ["Python_1", "Python_2", "AI_1", "AI_2", "Web", "Database", "Systems"]
hires = {
    "Ciara": ["Python_1"],
    "Peter": ["Python_2", "AI_1", "AI_2"],
    "Juan": ["Web", "AI_1", "AI_2"],
    "Jim": ["AI_1", "AI_2", "Systems"],
    "Jane": ["Python_2", "Database"],
    "Mary": ["Web", "Systems"],
    "Bruce": ["Systems", "Python_2"],
    "Anita": ["Web", "AI_1", "AI_2"]
}

problem = Problem()

# Define the variables (positions) and their domains (hires who can take the positions)
for position in positions:
    problem.addVariable(position, list(hires.keys()))

# Define the constraints
'''
IMPORTANT: Ciara only has the funds to hire 3 more people
She knows python and will fill one of the python positions
for this reason
'''
# Ciara is hardcoded for one of the python positions
def ciara_python_constraint(python_1):
    return python_1 == "Ciara"

# A person can only ocuppy one of the two available AI positions
def ai_constraint(ai_1, ai_2):
    return ai_1 != ai_2

# Jane is the only one qualified to ocuppy the DB position 
def db_constraint(database):
    return database == "Jane"

# Defines the amount of people Ciara can hire
def ciara_hiring_constraint(*positions):
    return len(set(positions)) <= 4

# Defines that a person can only occupy a max of two positions
def max_two_positions_constraint(*positions):
    hires_count = dict()
    for hire in positions:
        if hire in hires_count:
            hires_count[hire] += 1
        else:
            hires_count[hire] = 1
    return all(count <= 2 for count in hires_count.values())

# Define the constraint that an individual can only occupy the positions they are qualified for
def qualification_constraint(*args):
    for position, hire in zip(positions, args):
        if position not in hires[hire]:
            return False
    return True

# Add the constraints to the problem
problem.addConstraint(max_two_positions_constraint, positions)
problem.addConstraint(qualification_constraint, positions)
problem.addConstraint(ciara_python_constraint, ["Python_1"])
problem.addConstraint(ai_constraint, ["AI_1", "AI_2"])
problem.addConstraint(db_constraint, ["Database"])
problem.addConstraint(ciara_hiring_constraint, positions)

# Solutions
solutions = problem.getSolutions()

# Remove duplicates
unique_solutions1 = []
for solution in solutions:
    # For positions with the same qualifications, sort the individuals
    ai_positions = sorted([solution["AI_1"], solution["AI_2"]])
    solution["AI_1"], solution["AI_2"] = ai_positions[0], ai_positions[1]
    
    # Add to unique solutions if not present already
    if solution not in unique_solutions1:
        unique_solutions1.append(solution)

# Print the solutions by position
for i, solution in enumerate(unique_solutions1):
    print(f"Solution {i+1}:")
    for position, hire in solution.items():
        print(f"Position {position}: {hire}")
    print()

Solution 1:
Position AI_1: Anita
Position AI_2: Jim
Position Database: Jane
Position Python_1: Ciara
Position Python_2: Jane
Position Systems: Jim
Position Web: Anita

Solution 2:
Position AI_1: Jim
Position AI_2: Juan
Position Database: Jane
Position Python_1: Ciara
Position Python_2: Jane
Position Systems: Jim
Position Web: Juan



## Scenario 2
### Ciara and Juan become partners, with the additional funds they can now employ four more people but must employ another AI Engineer, so they need 2 Python Programmers, 3 AI Engineers, 1 Web Designer, 1 Database Admin, and 1 Systems Engineer.

This scenario will assume the following hiring conditions:

- Ciara and Juan will be considered for the positions they are qualified for.
- Solutions with Ciara OR Juan OR none of them are also going to be considered.

In [ ]:
from constraint import Problem

# Define the positions and the hires with their qualifications
positions = ["Python_1", "Python_2", "AI_1", "AI_2", "AI_3", "Web", "Database", "Systems"]
hires = {
    "Ciara": ["Python_1", "Python_2"],
    "Peter": ["Python_1", "Python_2", "AI_1", "AI_2", "AI_3"],
    "Juan": ["Web", "AI_1", "AI_2", "AI_3"],
    "Jim": ["AI_1", "AI_2", "AI_3", "Systems"],
    "Jane": ["Python_1", "Python_2", "Database"],
    "Mary": ["Web", "Systems"],
    "Bruce": ["Systems", "Python_1", "Python_2"],
    "Anita": ["Web", "AI_1", "AI_2", "AI_3"]
}

problem = Problem()

# Define the variables (positions) and their domains (hires who can take the positions)
for position in positions:
    problem.addVariable(position, list(hires.keys()))

# # Define the constraints
def python_constraint(python_1, python_2):
    return python_1 != python_2

def ai_constraint(ai_1, ai_2, ai_3):
    return ai_1 != ai_2 and ai_1 != ai_3 and ai_2 != ai_3

# Jane is the only one qualified to occupy the DB position 
def db_constraint(database):
    return database == "Jane"

# Defines the amount of people Ciara can hire
def hiring_constraint(*args):
    hires_set = set(args)
    if "Ciara" in hires_set and "Juan" in hires_set:
        return len(hires_set) <= 6
    elif "Ciara" in hires_set or "Juan" in hires_set:
        return len(hires_set) <= 5
    else:
        return len(hires_set) == 4

# Defines that a person can only occupy a max of two positions
def max_two_positions_constraint(*args):
    hires_count = dict()
    for hire in args:
        if hire in hires_count:
            hires_count[hire] += 1
        else:
            hires_count[hire] = 1
    return all(count <= 2 for count in hires_count.values())

# Define the constraint that an individual can only occupy the positions they are qualified for
def qualification_constraint(*args):
    for position, hire in zip(positions, args):
        if position not in hires[hire]:
            return False
    return True

# Add the constraints to the problem
problem.addConstraint(python_constraint, ["Python_1", "Python_2"])
problem.addConstraint(ai_constraint, ["AI_1", "AI_2", "AI_3"])
problem.addConstraint(db_constraint, ["Database"])
problem.addConstraint(hiring_constraint, positions)
problem.addConstraint(max_two_positions_constraint, positions)
problem.addConstraint(qualification_constraint, positions)

# Get and print solutions
solutions = problem.getSolutions()

total_combinations = 0
unique_solutions2 = []
for solution in solutions:
    # For positions with the same qualifications, sort the individuals
    ai_positions = sorted([solution["AI_1"], solution["AI_2"], solution["AI_3"]])
    solution["AI_1"], solution["AI_2"], solution["AI_3"] = ai_positions[0], ai_positions[1], ai_positions[2]

    python_positions = sorted([solution["Python_1"], solution["Python_2"]])
    solution["Python_1"], solution["Python_2"] = python_positions

    # Add to unique solutions if not already present
    if solution not in unique_solutions2:
        unique_solutions2.append(solution)
        total_combinations+=1


# Print the unique solutions by position
for i, solution in enumerate(unique_solutions2):   
    print(f"Solution {i+1}:")
    for position, hire in solution.items():
        print(f"Position {position}: {hire}")
    print()
    
    
# # Print solutions with specific hires
# solutions_count = 0

# for i, solution in enumerate(unique_solutions):
#     if "Ciara" in solution.values():
#         print(f"Solution {i+1}:")
#         for position, hire in solution.items():
#             print(f"Position {position}: {hire}")
#         print()
        
#         solutions_count += 1

# print(f"The number of solutions is: {solutions_count}")



In [ ]:
# Initialize a dictionary to store the counts for each person
person_counts = dict()

# Loop over the unique solutions
for solution in unique_solutions:
    # Get the set of hires in each solution
    hires_set = set(solution.values())
    # Loop over the hires in the set
    for hire in hires_set:
        # Increment the count for the hire by one
        person_counts[hire] = person_counts.get(hire, 0) + 1

# Define a function to extract the count from an item
def get_count(item):
    return item[1]

# Sort the person counts by ascending order of values
person_counts = dict(sorted(person_counts.items(), key=get_count))

print(f"There is a total of {total_combinations} hiring combinations")
print()
# Print the counts for each person
for person, count in person_counts.items():
    if count >= 176:
        print(f"{person} appears in ALL solutions")
    else:
        print(f"{person} appears in {count} solutions.")
